# Load From a VCF File Example

A central point to the SGkit API is the Genotype Call Dataset. This is the data structure that most of the other functions use. It uses [Xarray](http://xarray.pydata.org/en/stable/) underneath the hood to give a programmatic interface that allows for the backend to be several different data files.

The Xarray itself is *sort of* a transposed VCF file.

For this particular example we are going to go from a VCF file to the Genotype Call DataSet. 

**Please note that in the real world you *should not* read in your VCF files like this, but instead use the functionality in sgkit to go from a VCF to a Zarr file.** 

We are starting from the VCF file in order to give a conceptual understanding of the data structure itself.

In [1]:
import numpy as np
import zarr
import pandas as pd
import dask.array as da
import allel
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

## Prep Work - Install Packages

SGKit is still under rapid development, so I'm installing based on a commit. 

In [2]:
#! pip install git+https://github.com/pystatgen/sgkit@96203d471531e7e2416d4dd9b48ca11d660a1bcc

### Install PyVCF

You'll need to install PyVCF, samtools and tabix in order to run this example as is. 

PyVCF needs to be in the same kernel in order to use it, but tabix can be installed anywhere.

In [3]:
# Or install to your existing environment
# ! conda install -c bioconda -c conda-forge -y pyvcf samtools tabix


# Uncomment these to create a new conda environment and install these packages
# If you create a new environment you will have to switch your jupyterhub kernel
# ! conda create -n samtools -c bioconda -c conda-forge -y samtools pyvcf samtools tabix
# ! conda activate samtools 

# ! tabix -h ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20100804/ALL.2of4intersection.20100804.genotypes.vcf.gz 2:39967768-39967768 > chr2.vcf
# ls -lah chr2.vcf

## Grab Some Data

We're going to grab a small subset of a VCF file from the [1000 Genomes Project.](https://www.internationalgenome.org/faq/how-do-i-get-sub-section-vcf-file/). We're only going to grab 3 calls, which is fine for our purposes.

These calls are also already biallelic. I cheat. ;-)

In [4]:
# I couldn't run this from jupyterhub but needed an actual terminal
#! conda activate samtools 
#! tabix -h ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20100804/ALL.2of4intersection.20100804.genotypes.vcf.gz 2:39967768-39967800 > chr2.vcf
#! conda deactivate
# ls -lah chr2.vcf

In [5]:
import vcf
import os

Let's write up a quick, *not to be used in the real world*, parser to grab data about the variants.

* Variant Contig Names - A unique list of all the chromosomes and contigs
* Variant Contig - an index of the variant_contig_names list.
* Variant Position - Position on the chromosome
* Variant Reference and Alternate
* Samples
* Genotype calls per sample - with missing encoded as -1



In [6]:
vcf_reader = vcf.Reader(open('/home/jovyan/chr2.vcf', 'r'))

# I already know these come from chr2
# but let's grab them anyways
variant_contig_names = []

variant_chrom = []
variant_position = []
variant_alleles = []
variant_contig = []

sample_id = []
call_genotype = []

count = 0

for record in vcf_reader:
    
    chrom = str(record.CHROM)
    if chrom not in variant_contig_names:
        variant_contig_names.append(chrom)
        
    # Grab the index of the contig
    variant_contig.append(variant_contig_names.index(chrom))
    
    # Get the variant data
    # I'm cheating and only getting the first alternate. In the real world you would filter for biallelic variants.
    variant_alleles.append([str(record.REF), str(record.ALT[0])])
    variant_position.append(record.POS)
    
    # the sample records is an object that has call data       
    samples = record.samples
    
    # Grab the sample names
    if count == 0:
        for sample in samples:
            sample_id.append(sample.sample)
    
    # Grab the call data for each sample for the variant
    variant_genotypes = []
    for sample in samples:
        # If its missing encode as -1, -1
        if sample['GT'] == './.':
            variant_genotypes.append([-1, -1])
        else:
            GT = sample['GT'].split('|')
            variant_genotypes.append([int(GT[0]), int(GT[1])])
    
    call_genotype.append(variant_genotypes)
    count = count + 1

## Convert to Numpy

Now that we have our data, we need to prepare for our XArray dataset by converting these to Numpy arrays.

If you're wondering how I know what these are you can check out the `sgkit.api.create_genotype_call_dataset`. The exact functions are `check_array_like` and make sure that these are numpy arrays of a particular type.

```
check_array_like(variant_contig, kind="i", ndim=1)
check_array_like(variant_position, kind="i", ndim=1)
check_array_like(variant_alleles, kind="S", ndim=2)
check_array_like(sample_id, kind="U", ndim=1)
check_array_like(call_genotype, kind="i", ndim=3)
```

In [7]:
sample_id = np.array(sample_id, dtype='U')
variant_position = np.array(variant_position, dtype='i')
variant_alleles = np.array(variant_alleles, dtype='S')
variant_contig_names = np.array(variant_contig_names, dtype='S')
variant_contig = np.array(variant_contig, dtype='i')

### Understanding Variant Contig and Variant Position

The Genotype Call Xarray dataset is meant to be able to incorporate multiple chromosomes.

Let's say we have variant calls from chrs 1 and 2, which we read into an array `['chr1','chr2']`.

In [8]:
import pandas as pd

In [9]:
contigs = ['chr1', 'chr2']
    
df = pd.DataFrame({
                    'variant_contig_index': [0, 0, 1, 1],
                    'variant_position': [1, 2, 1, 2],
                    })
df

,variant_contig_index,variant_position
0,0,1
1,0,2
2,1,1
3,1,2


The Xarray dataset looks like the dataframe above. 

When we initialize the Xarray dataset we will give it a list of contigs (or chromosomes). We don't need to explicitly list the contig per position because we can calculate this based on the contig index.

**Contig**: `contigs[row['variant_contig_index']]`

**Position**: `row['variant_position']`

In [10]:
def return_contig(row):
    return 'Chr: {chr} Pos: {pos}'.format(chr=contigs[row['variant_contig_index']], pos=row['variant_position'])

df['description'] = df.apply(lambda row: return_contig(row), axis=1)

df

,variant_contig_index,variant_position,description
0,0,1,Chr: chr1 Pos: 1
1,0,2,Chr: chr1 Pos: 2
2,1,1,Chr: chr2 Pos: 1
3,1,2,Chr: chr2 Pos: 2


### Genotype Calls

If we've done our work right we our genotypes should have the shape: `[DIM_VARIANT, DIM_SAMPLE, DIM_PLOIDY]`, meaning the first axis is the number of variants, the second the number of samples, and the third the ploidy. In our case we are working with diploid alleles.

Our genotype array has this structure:

```python
genotypes = [

    # Outermost array should have a length = the number of variants
    
    # variant chr 1 position 1
    [
        # Per variant we should have an array length = number of samples
        
        # sample 1 
        # Per sample we should have an array length = number of alleles
        [call, call],
        
        # sample 2
        [call, call]
    ],
    
    # variant chr 1 position 2
    [
        # sample 1 
        [call, call],
        # sample 2
        [call, call]
    ],
    
]
```

In [11]:
call_genotype = np.array(call_genotype, dtype='i')
call_genotype.shape

(3, 629, 2)

This is correct! We have 3 variants, 629 samples, and diploid alleles.

## Convert to Genotype Call Dataset

Finally! Let's convert this to the Genotype Call Dataset!

In [12]:
variant_alleles

array([[b'T', b'A'],
       [b'G', b'C'],
       [b'C', b'T']], dtype='|S1')

In [13]:
import sgkit

genotype_xarray_dataset = sgkit.api.create_genotype_call_dataset(
    variant_contig_names = variant_contig_names,
    # Since we know these are all from the same chromosome we could just calculate this on the fly as a np array of zeros
    #variant_contig = np.zeros(len(variant_position)),
    variant_contig = variant_contig,
    variant_position = variant_position,
    variant_alleles = variant_alleles,
    sample_id = sample_id,
    call_genotype = call_genotype,
)

In [14]:
genotype_xarray_dataset

<xarray.Dataset>
Dimensions:             (alleles: 2, ploidy: 2, samples: 629, variants: 3)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant/contig      (variants) int32 0 0 0
    variant/position    (variants) int32 39967768 39967778 39967793
    variant/alleles     (variants, alleles) |S1 b'T' b'A' b'G' b'C' b'C' b'T'
    sample/id           (samples) <U7 'HG00098' 'HG00100' ... 'NA20828'
    call/genotype       (variants, samples, ploidy) int32 0 0 0 0 1 ... 0 0 0 0
    call/genotype_mask  (variants, samples, ploidy) bool False False ... False
Attributes:
    contigs:  [b'2']

## Done!

Now we have our Xarray dataset that we can use with the rest of Sgkit!